# Hedgehog
*(macros here; code in appendices)* 
$%Discrete Model$
$\newcommand{\nbuff}[]{n_b} $
$\newcommand{\nact}[]{n_a} $
$\newcommand{\nres}[]{n_r} $
$\newcommand{\state}[1][t]{\mathbf{x}(#1)} $
$\newcommand{\action}[1][t]{\mathbf{u}(#1)} $
$\newcommand{\arrival}[1][t]{\mathbf{a}(#1)} $
$\newcommand{\constituency}[]{\mathbf{C}} $
$\newcommand{\processing}[1][t]{\mathbf{B}(#1)} $
$\newcommand{\cost}[]{\mathbf{c}} $
$\newcommand{\costfn}[]{f_{\cost}} $
$%Fluid Model$
$\newcommand{\flstate}[1][t]{\mathbf{q}(#1)} $
$\newcommand{\flactivitycumul}[1][t]{\mathbf{z}(#1)} $
$\newcommand{\flactivitycumulfree}[]{\mathbf{z}} $
$\newcommand{\flactivity}[1][t]{\mathbf{\zeta}(#1)} $
$\newcommand{\flarrival}[]{\mathbf{\alpha}} $
$\newcommand{\flprocessing}[]{\mathbf{B}} $
$%Workload$
$\newcommand{\nwork}[]{n_w} $
$\newcommand{\workload}[]{\mathbf{\xi}} $
$\newcommand{\workloadmat}[]{\mathbf{\Xi}} $
$\newcommand{\workloadmathat}[]{\hat{\mathbf{\Xi}}} $
$\newcommand{\bottleneck}[]{\mathbf{\nu}} $
$\newcommand{\worktime}[1][]{\mathbf{w}#1} $
$\newcommand{\loadbalance}[]{\mathbf{\rho}} $
$\newcommand{\effcost}[]{\mathbf{\bar{c}}} $
$\newcommand{\effcostfn}[]{f_{\effcost}} $
$\newcommand{\monotone}[]{\mathbb{W}^+ } $
$\newcommand{\idledirection}[]{\mathbf{v}^*}$
$\newcommand{\idlingset}[]{\mathbb{K}}$
$\newcommand{\nonidlingset}[]{\mathbb{S}}$
$%Hedging$
$$\newcommand{\hedging}[]{\beta^*}$$
$%SafetyStocks$
$$\newcommand{\safetystockpenalty}[]{\mathbf{\psi}}$$
$%Generic$
$\newcommand{\policy}[]{\phi} $
$\newcommand{\T}[]{\top} $
$%Derived Symbols$
$\newcommand{\statetzero}[]{\mathbf{x}(0)} $

# 1 . Whats the Problem?

A network is composed of three elemets: *buffers*, *actions* and *resources*.
* A *buffer* is a queue, whose state is the number of items in it.
* An *action* changes the state of certain buffers.
* A *resource* is an agent that can perform certain actions, one per time step.

The topology and dynamics of the network is specified by two matrices and a vector:
* The *constituency matrix* indicates which resources can perform which actions. No two resources can perform the same action.
* The *buffer processing matrix* indicates what the effect of certain actions will have at time $t$ on the buffer states
* The *arrivals vector* indicates what new items enter each buffer at time $t$, from outside the system.

The evolution of the states $\state$ over time is a simple Markov Decision Process, that is also a Controlled Random Walk (CRW) - a random walk, where one element of stochasticity can be controlled by actions $\action$. 

$$ \state[t+1] = \state + \processing[t+1]\action + \arrival[t+1]$$ 

where $\processing$ is the processing matrix and $\arrival$ is the arrivals vector.

Our policy $\policy$, which is a mapping from states to actions, is the controlled part of the random walk: $\policy : \mathbb{X} \rightarrow \mathbb{U}$.

$$ \state[t+1] = \state + \processing[t+1]\policy(\state) + \arrival[t+1]$$ 

#### Objective

We define a cost at each time step, which is a function of the state of all buffers. In our case this is a linear function (i.e. each buffer has a fixed cost per item in queue).

$$\text{step cost} = \costfn(\state) = \cost^\T\state$$

We then define a value function $v^\policy(\mathbf{x}')$, which is the expected sum of discounted future cost under a policy function $\policy$, given the current state. By minimising this functional, we will find our optimal behaviour - the $\policy^*$ function.

$$\policy^* = \text{argmin}_{\policy} v^\policy(\mathbf{x}') = \text{argmin}_{\policy} \mathbb{E}[\sum_{t=0}^{\infty}\gamma^tf_c(\state) |  \state[0] = \mathbf{x'}]$$




# 2. How to solve it?

The solution to the CRW functional is intractable. We solve with a number of approximations, and find a solution that is asymptotically optimal under heavy-load and heavy-traffic assumptions.

* **heavy load** - new items arrival rate is close to the slowest processing rate. 
* **heavy traffic** - there are always enough items in the buffer to do all actions.

The algorithm to solve our problem is outlined below...

### 2.0. Spoilers
**TL;DR:** *Our approximately optimal policy (under heavy-load, heavy-traffic assumptions) is simple: critical resources (bottlenecks) can never idle, except in the special case where them idling helps us strategically lower the cost in the long run. Our online algorithm finds these non-idling bottlenecks, then runs an optimisation to find activities by greedily minimising the cost. A penalty term is added to this optimisation so no buffers are _completely_ empty, so all activities are possible ('safety stocks'). Our final activities can be translated into actions probabilistically over a time period.*

1. **Find the set of non-idling bottleneck ($\nonidlingset$)**
    1. **We transform our problem into a new 'workload' space.** From a buffer-state space, we move to a space where each dimension corresponds the amount of time each resource needs to work to drain the network. The idea is that we can approximate our problem by only looking at the resources which take the longest to drain the network - the bottlenecks. This reduces the dimensionality and provides with a good physical intuition with how policies move us in the space: 
        * *Idling a single resource, moves you up a coordinate axis (+ve).*
        * *Working a single resource, moves you down a coordinate axis (-ve).*
    2. **We find the nearest point in the 'monotone region'.** In our workload space, we can plot our effective cost - it looks like a piecewise linear function made of intersecting hyperplanes. In this space, there is a region where moving towards the origin along any axis is guaranteed to lower the effective cost. This is the monotone region.
        * *Inside the monotone region our optimal policy is pretty obvious -  work all resources all the time!*
        * *Outside this region, a step to the origin may actually increase the cost - lets idle towards the monotone region!*
    3. **We calculate a hedging distance, from the 'monotone region'.** The asymptotically optimal policy in terms of total cost in a _non-stochastic_ environment, is  to move to the monotone region by idling certain resources, and once you're there, work all resources all the time. In fact, to compensate for stochastic fluctuations, we actually only need to idle until we get to a fixed distance from the monotone region, before we work all out. This fixed distance is called the hedging distance and it differs depending on which face of the region you are approaching. Our policy now becomes: 
        * *Inside the ~~monotone region~~ hedging region -  work all resources all the time!*
        * *Outside this region - lets idle towards the hedging region*!
4. **Calculate the safety stock parameter ($\safetystockpenalty$)** Our algorithm relies on an assumption that resources can always work when they are told to - i.e. the buffers are never fully empty. However, this is not guaranteed by our algorithm. We calculate a penalty to add to the cost when our resources approach starvation to compensate for this.
5. **Choose activity rates (activities) to minimize the step cost, given non-idling bottleneck constraints and safety stock penalty$\safetystockpenalty$** This is simply the result of the constrained activation, where we greedily solve to minimise the step cost given our non-dling constraints and safety stock penalty vector.
6. **Translate these continuous activity rates to discrete actions over a given time period** Each activity (which is a real number) is converted to a discrete series of actions over a time period, probabilistically.

Now we move to the model in greater detail.


### 2.1 The Fluid Model

Our fundamental model is discrete, which makes it hard to solve. We start by approximating our CRW by a fluid model. This can be considered approximation of the network on very long timescales - "zooming out" on the time axis. The implications of this are:
* our time is now continuous
* our state is now continuous: $\state \rightarrow \flstate$
* our arrivals can be replaced by an expected arrival rate: $\arrival \rightarrow \flarrival$
* our processing matrix can be replaced by an expected processing matrix: $\processing \rightarrow \flprocessing$
* our actions are now an action rate or 'activity': $\action \rightarrow \flactivity$
* $\therefore$ our system is now deterministic


Note that to define the instantaneous activity, we can start by defining the 'cumulative activity' $\flactivitycumul$, that represents how much each action has been worked on by time $t$. By taking the derivative of this with respect to time, we get our instantaneous activity  $\flactivity = \dfrac{d\flactivitycumul}{dt}$ . It is assumed $\flactivity \in \mathbb{U} $ for all $ t$.

Our instantaneous dynamics are now:

$$ \dfrac{d\flstate[t]}{dt} =  \flprocessing\flactivity + \flarrival$$ 

Or, in terms of cumulative change:

$$ \flstate = \flstate[0] + \flprocessing\flactivitycumul + \flarrival t$$ 

### 2.2. Workload Space

Even with these continuous dynamics, the solution is still intractable. Our first move is to transform the problem into a new space, in which the problem is expressed in terms of "bottlenecks". 

These bottlenecks are the resources of the network that most greatly affect the draining time of the network, and are critical to the dynamics of the system. Our state space transformation will move us from buffer space (where each axis marks the number of items in a specific buffer) to workload space (where each axis marks the amount of time a specific resource must work in order to drain their buffers).

By only choosing the most loaded resources to be axes in this space, we approximate our problem by reducing its dimensionality. This is analagous to finding the k-leading singular values in a large matrix problem, and solving problems in this reduced subspace. Note that, since we often have more buffers than resources, workload space is already a reduced dimensionality representation, but the key approximation to our system comes from not considering resources which are not the bottlenecks of our problem. 

Note that the total activities that each bottleneck should perform can be collected into a vector, called the worktime vectors $\worktime[(t)]$. As such, we seek a transformation from $\state \rightarrow \worktime[(t)]$, which is linear $\worktime[(t)] = \workloadmat \state$. But how do we find which resources are bottlenecks? And how do we find the transformation $\workloadmat$  to go from buffer state to worktime per resource? 

We consider a thought experiment: 

------

#### Thought Experiment

*Imagine the network starting at time $t=0$ with a state of $\mathbf{q}_1$.  How would we choose activities for all the resources to minimise the amount of time to drain the network from this state to a lower state $\mathbf{q}_2$, assuming **no new arrivals**?*

This is a constrained optimisation problem. We seek to find the minimal $T$, subject to a number of physical constraints:

1. The change in fluid of the system occurs over time $T$. In terms of the cumulative change dynamics equation:
$$ \mathbf{q}_2 = \mathbf{q}_1 + \flprocessing\flactivitycumul[T] + 0$$ 

2. Secondly each resource $r$ can not have worked more than $T$ time in total, across all it's activities. Since no two resources work on the same action, each row of $\constituency$ multiplied by the cumulative activity, $\flactivitycumul$, picks out the sum of activities that the resource has done. This must be less than $T$. As a matrix equation:

$$\constituency\flactivitycumul[T] \leq \mathbf{1}T =  \mathbf{T}$$


3. Finally it is not physical to have a negative activity, nor indeed negative time.
$$ \flactivitycumulfree \geq \mathbf{0}$$
$$T \geq 0$$

-----

#### Thought Experiment: Our Optimisation

Our constrained optimisation is thus:
$$
 \begin{array}{ccccccc|ccc}
\text{Primal} &&&&  \text{Dual}  &&&& \textit{Dual Rewritten}\\
\hline
\min_{\mathbf{z}, T} T     &&&& \max_{\workload, \bottleneck} \workload^\T( \mathbf{q}_1 -  \mathbf{q}_2) &&&& \max_{\workload, \bottleneck} \workload^T\flarrival \\
\text{ s.t.}               &&&& \text{ s.t.}  &&&& \text{ s.t.} \\
\mathbf{q}_2 = \mathbf{q}_1 + \flprocessing \flactivitycumulfree &&&&   \flprocessing^\T \workload + \constituency^\T\bottleneck \geq \mathbf{0}  &&&& - \flprocessing^T \workload - \constituency^T\bottleneck \leq \mathbf{0}\\
\constituency \flactivitycumulfree \leq \mathbf{T} &&&& - \mathbf{1}^\T\bottleneck + 1 \geq 0 &&&& \mathbf{1}^T\bottleneck \leq 1 \\
\flactivitycumulfree \geq \mathbf{0} &&&& \bottleneck \geq \mathbf{0} &&&&  -\mathbf{I} \bottleneck \leq \mathbf{0}\\
T \geq 0 &&&&\\
\end{array}
$$

Note that in the last case, we have let  $\mathbf{q}_1 = \flarrival$, $\mathbf{q}_2 = 0$ - i.e we drain to empty from an initial state of $\flarrival$. We can thus solve this succinct linear optimisation to find our $\workload$.

#### The Workload Matrix and Relaxed Workload Matrix
Of course the solution to the optimisation will only return the workload vector $\workload$ corresponding to the slowest resource - the biggest bottleneck.  We actually want the vectors corresponding to many more resources!

One way of finding these is simply to eliminate that resource for the problem and recalculate, though in practice it is simpler to note that that constraints can be written as a block matrix of constraints of form $\mathbf{Ax} \leq \mathbf{b}$:


$$\Big[ \begin{array}{c c}
-\flprocessing^\T & -\constituency^\T \\
\hline
\mathbf{0} & -\mathbf{I} \\
\hline
vec(0)^\T & vec(1)^\T \\
\end{array} \Big]
\Big[  \begin{array}{c}
\workload \\
\hline
\bottleneck \\
\end{array}\Big]
\leq
\Big[
\begin{array}{c}
\mathbf{0} \\
\hline
\mathbf{0}  \\
\hline
1 \\
\end{array} \Big]
$$

These constraints define a polyhedron in workload space, and given our maximisation objective $\workload^\T\flarrival$ is linear - all maximisations in this space lie along the vertices of this polyhedron. We therefore find the vertices of this polyhedron using a vertex enumeration library, each giving us a bottleneck $\bottleneck$ and workload vector $\workload$.

Each $\bottleneck$ indicates to us which resource in particular the workload vector refers to, since it is a vector that lies on the simplex over resources. It will have non-zero value on the resource to which it's $\workload$ refers.

We can also collect our workload vectors $\workload$ into rows of a matrix $\workloadmat = [\workload_1, ..., \workload_{\nres}]^\T$, ordered decreasingly by their the time it takes for them to drain an arrival at a single time step - "the load balancing fraction": $\loadbalance_i = \workload_i^\T \flarrival$. This $\loadbalance_i$ is necessarily fraction, because if it is greater than one then clearly the buffer state will grow unboundedly at that resource and the network can never be drained. Note that our "heavy load" approximation can now be well defined: $\max_i \loadbalance_i \approx 1$


Now, having a workload matrix $\workloadmat = [\workload_1, ..., \workload_{\nres}]^\T$ with rows sorted largest to smallest by $\loadbalance$, our approximation is to keep only the resources that appear to be bottlenecks - those with load balancing fraction greater than some constant: say $\loadbalance_i > 0.7$. This gives our new "relaxed" workload matrix:

$$\workloadmathat =  \workloadmat_{[:\nwork, :]} = [\workload_1, ..., \workload_{\nwork}]^\T;  \nwork \leq \nres $$



### 2.3. Workload Policies

#### The Problem in Workload Space

Our problem has been transformed into the workload space ($\workloadmat$) and relaxed ($\workloadmathat$), such that we are now addressing the problem in terms of $ \worktime = \workloadmathat \state$. This worktime $\worktime$ is the time that each of our chosen bottlenecks needs to work to drain their "fluid buffers".  In this space, we can define our problem with a new "effective cost" function $\effcostfn$, defined in relation to our original cost. 

$$\effcostfn(\worktime) = \min_x \cost^\T \state\\ \text{ s.t. } \\\workloadmathat \state = \worktime $$

We take a minimum over $\state$ because there may be multiple different $\state$ states that have the same worktime $\worktime = \workloadmathat \state$, and we define our effective cost as the minimum of these possible states, since this minimum will correspond to the cost associated with the buffer states affected by our bottlenecks (those buffers which arent affected by our bottlenecks).

This minimisations can once again be expressed in terms of a dual optimisation, which is a simple linear function subject to some inequality constraints.

$$
 \begin{array}{ccccccc}
\text{Primal}  &&&&  \text{Dual}  \\
\hline
\min_{\state} \cost^\T\state            &&&& \max_{\effcost} \effcost^\T \worktime \\
\text{ s.t.}                       &&&& \text{ s.t.} \\
\workloadmathat \state = \worktime &&&&  \workloadmathat^\T \effcost \leq \cost \\
\state \geq 0                      &&&&  \\
\end{array}
$$

Once again, the inequality constraints define a polyhedron and the solution to which $\effcost$ maximises the cost is the vertex of this polyhedron, picked out by the slope of the linear function (in this case our $\worktime$). We could find all these $k$ different possible solutions $\{\effcost_k\}$ by vertex enumeration if we wished, or we could simply perform the above optimisation to find the exact value of the effective cost function $\effcostfn$ at our current $\worktime$

Eithey way our effective cost function $\effcostfn$ in $\worktime$-space thus clearly a piecewise linear function, made up as a maximum over k overlapping hyperplanes, defined by vectors $\{\effcost^k\}$.

$$\effcostfn(\worktime) = \max_k {\effcost^k}^\T \worktime$$

In [ ]:
# An Example Effective Cost Surface 
%matplotlib notebook
cbar1 = np.array([-2, 2])
cbar2 = np.array([1, 1.5])
cbar3 = np.array([3,-3])
# cbar4 = np.array([0,-5])
cbars = [cbar1, cbar2, cbar3]

f_cbar = lambda w : np.max(np.array(cbars) @ w, axis=0)

plot_piecewise_workspace2D(cbars, [])

#### Our Policies in Workspace
We have now:
* an approximation to map our buffer state process $\state$ into a (lower-dimensional) worktime state process $\worktime [(t)]= \workloadmathat \state$
* an approximation to the cost function in this space $\effcostfn(\worktime) = \max_{k} {\effcost^k}^\T \worktime$

In this new workload space, the coordinate axes have a clear interpretable meaning - each axis represents the time a particular resource must work to drain its buffers (assuming no new arrivals). Navigating the workspace can now easily be done by  choosing actions for particular resources:
* Moving along up a coordinate axis (increasing our worktime for one resource) corresponds to idling that particular resource as arrivals continue.
* Moving down an axis to the origin implies servicing a buffer fast than arrivals appear. 

However we still have to find our optimal policy to move about in this workload space. 

*Q: How do we define the policy what to idle and not idle, to optimise the cost?*

A: In this new space, the components of $\effcost^k$ are particularly important, as they indicate the change in effective cost upon moving along a coordinate axis in worktime. In a region where all components are positive, our optimal policy is well defined - step towards the origin $\implies$ non-idle all bottlenecks. This region is knows as the monotone region:

$$\monotone = \{\worktime : \effcostfn(\worktime') \geq \effcostfn(\worktime) \text{ for all } \worktime' \geq \worktime,\worktime' \in \mathbb{W}  \}$$

Outside of this region, it is not clear whether a random step of idling on not idling will necessarily lower the cost - especially in high dimensions. Our optimal policy is to get to the monotone region as quickly as possible, before going flat-out on not-idling. This can be done by idling a single bottleneck (which moves us along a coordinate axis) into the monotone region.  Our aim will be to find the closest face of the monotone region, and allow our final optimisation to idle in that direction, while constraining all other bottlenecks to non-idle.

In [ ]:
w = np.array([5, 0.3])
load = np.array([0.7, 0.9])


origin = np.zeros_like(w)
main_path = get_full_paths(w, cbars, load)
_, w_star, l = main_path[0] # note: w_star only first 
    
paths = [
    [(w, origin , drift_vec(-w, load))],
    [(w, w_star, l), (w_star, origin, drift_vec(-w_star, load))],
    main_path
]

annotations = get_w_star_annotations(w, w_star)
annotations += get_path_annotations(paths)
annotations += get_load_annotations(w, load)
plot_piecewise_workspace2D(cbars, annotations)

In [ ]:
plot_paths(cbars, paths)

#### Finding the Nearest Face to the Monotone Region

As it stands, we know we occupy a point in workspace, and want to get to the montone region but we have no analytic idea where the monotone region exists. However we know that if the monotone region is accessible by idling, the point of minimum cost for all points that are reachable by idling *must be in the monotone region*. 

$$\worktime^* = \text{argmin}_{\worktime'} \effcost^\T\worktime' \text{ s.t. } \worktime' \geq \worktime,  \implies \worktime^* \in \monotone$$


----
*Quick Argument*
1. Assume the $\worktime^*$ that minimises $\effcostfn = \effcost^\T\worktime $ is not in the monotone region.
2. Since the cost function is convex along each coordinate direction/axis, if we consider a step along any coordinate axis:
    * If any step lowers the cost, the point is not the minimum.
    * If no steps lower the cost - the point must be in the monotone region.
3. Proof by contradiction, 1. must be False. 
----
Knowing this proof, we simply find $\worktime^*$ by performing this exact minimisation, solving with the above LP (or its dual). 

Our result is $\worktime^*$ gives us a direction $\idledirection$ in which we may idle, and the components of these vector which are non-zero are those resources which may idle. All other components of this vector (which are zero) correspond to resources which must non-idle and be constrained to do so. In this way we define two sets of bottlenecks: the idling set $\idlingset$ and the non-idling set $\nonidlingset$.

$$\idledirection := \dfrac{ \worktime^* - \worktime}{| \worktime^* -\worktime|} $$
$$\idlingset := \{i: \idledirection_i > 0\}$$
$$\nonidlingset := \{i: \idledirection_i = 0\}$$
$$\text{where}$$
$$\idlingset \cap \nonidlingset = \mathbb{0} \text{ and } |\idlingset| + |\nonidlingset| = \nwork $$

We have now found our non-idling set  $\nonidlingset$, which we will be constrained to work, when we solve for our optimal actions, back in buffer space.

### 2.3. Hedging

We have now translated the random walk of our buffer state process $\state$ into an approximate lower-dimensional worktime process $\worktime[(t)]$.
This worktime process is also controlled random walk, where our controlled decision is whether to work or idle each bottleneck resource. We have seen that our optimal policy in this workload space is to nonidle all resources in the monotone region $\monotone$, and outside $\monotone$, to idle $\idlingset$ towards it in the $\idledirection$ direction. In buffer space, this policy corresponds to constraining all nonidling $\nonidlingset$ to non-idle, while solving a greedily minimising the cost. (This should still take us towards the monotone region).

However, we have neglected a large part the model so far - the stochasticity of all the processes. Given our arrivals and servicings are stochastic, any decisions to work or idle will be subject to noise, bumping us randomly through the workspace. Given the stochasticity of the process, it may not be worth idling all the way until we are in the monotone region before we set all resources to work - instead we might just need to idle to fixed distance from the nearest face before we go 'all out' on our nonidling.

Indeed this is what Sean has proved in his book - that the asymptotically optimal policy is simply an affine translation (ie a parallel shift) of each of the faces of the monotone region. The distance that each face $i$ must be shifted is called the hedging distance $\hedging_i$. Since the number of faces of $\monotone$ grows combinatorially with dimension of the space, we calculate each hedging distance online. This means we only refer to the one hedging distance we are interested in at a time, and can drop our index $i$: $\hedging_i \rightarrow \hedging$.  

#### Hedging Policy
Our policy now becomes:
* If we are in $\monotone$ - constrain all bottlenecks to non-idle
* If we are within $\hedging$ of the nearest face of $\monotone$ - constrain all bottlenecks to non-idle
* Else, act as before - allow some bottlenecks to idle, to move us towards the monotone/enlarged region

An example image is shown below

In [ ]:
x_lim = [-2, 20]
y_lim = [-2, 20]

monotone_point = [4,4]

# find gradients of intersection w monotone region -> define affine hedging regions
axis_beta_stars = [.7, -1.5]
grads = get_grads_of_intersection(cbars, monotone_point) 
annotations = get_hedging_region_annotations(grads, axis_beta_stars, x_lim, y_lim)

noise = .25
drift = -(1 - load)
grad, beta_star = grads[-1], axis_beta_stars[-1] # lower surface gradient and beta star
drift_g = gradient_to_drift_scale(grad, drift) # make a drift vector out of gradient 

start = [15,  15 * grad] # start on intersection
seconds = 40
noise_per_second = 5

noisy_paths = generate_noisy_paths(start, drift_g, grad, noise, seconds, noise_per_second)

path_annotations = get_noisy_path_annotations(grad, drift_g, [beta_star, 0], noisy_paths)

annotations += path_annotations
annotations += get_load_annotations([0,0], load)

plot_piecewise_workspace2D(cbars, annotations, contour_only=True, x_lim=x_lim, y_lim=y_lim)


#### Hedging Intuition

It can be difficult to intuit why stochasticity should increase a hedging distance. To illustrate the need for hedging, first consider starting at $\worktime^*$, and non_idling continuously towards the origin. For the sake of simplicity, we assume the direction in which we are non idling - the drift - is the same as the gradient.

In a deterministic environment, this paths is a straight line. In a stochastic environment, where spherical noise is drawn and added to the state at each step, we obtain a random walk. However, under our policy, whenever we move below the gradient slope, our y_component idles - resulting in a _reflected random walk_ along the gradient axis.

Samples of possible paths are shown below, with the noiseless walk, random walk, and reflected random walk plotted together.

In [ ]:
plot_3_example_walks(cbars, load)

The fact that our random walk is reflected along a the intersecting line of two hyperplanes ( called it the 'reflection axis') clearly affects the distribution of possible states that the random walk can be in. Previously, the mean of the random walk would have been the drift - however now that the mean of the walk is shifted, away from the reflection  axis. This change (from random walk to reflected random walk) is visible below, using a sample of 1000 random walks, with the quartiles and means plotted. 


In [ ]:

grad = grads[-1] # lower surface gradient and beta star
drift = gradient_to_drift_scale(grad, drift) # make a drift vector out of gradient 
# drift = - (1 - load)

plot_distributions_walks(cbars, drift, 0.25, 1)

From the plots above it is clear that to minimise the total cost, we will need to shift the new trajectory of states vertically so the more states now lie along the valley defined by reflection axis. However, it should be noted, not all states are weighed equally - cost is discounted with time! This brings us to our final right most plot: this is the distribution of all states encountered across all sample paths, with each step weighted by its discount factor. This is the 'cloud' that we want to shift to minimise our future cost.

With this in mind we can intuit the factors that impact our shift (our hedging):
* The relative steepness of the two faces in question along the axis parallel to the reflection axis:
    * If monotone/non-monotone is high, hedging is high
    * If monotone/non-monotone is low, hedging is low
* The drift direction of the random walk:
    * If drift is away from the axis -> More hedging
    * If drift is towards than the axis -> Less Hedging 
* The noise of the random walk:
    * A noisier walk -> A bigger step -> More hedging
* The discount factor:
    * A discount factor of 0, suggests you should start at the minimum -> No hedging
    * A discount factor of 1, suggests you should factor in the very long term effects of drift -> High Hedging

These are the 4 factors that impact hedging.

#### Hedging In Practice

#### Hedging Assumptions


It is worth noting the assumptions that lead to our hedging calculation. First of all, it is clear from our graphs that we assume.

### 2.4 Safety Stocks

The first thing is to consider that, under our current policy, whene
Consider a guided random walk along the cost surface, starting at the edge of the monotone region, in which we consistently guided towards the origin. 



* solve linear program to get to nearest face.
* but we dont want to move exactly to the nearest face, because the random walk will push us around. its a waste. at an optimum point we just start moving to origin, because the random walk will push us around the steep surface.
    * can we have a negative beta? ie move - into the monotone region because the final surface is too steep
    * idling corresponds to moving along a coordinate axis, but doesnt the rest move (non-idle?) so actual policy is not that straight line to the axis?
* we calculate that distance with a hedging parameter based on the cost profile and the statistics of the random walk
* this distance (an affine enlargement of the monotone region) indicates our policy: if outside region -> idle towards it. otherwise, move to origin by non-idling on everything!

### Safety Stocks
* An approximation thats made is that, when non-idling, work can always be done on the buffers. 
* in a fluid model thats not a problem, but with discrete packets, cant do work on an empty buffer
* we add constraints to indicate which buffers cannot be empty, and find a penalty function on emptying buffers. 
* This will return a vector which will be added to our cost.

### Feedback Policy
We have obtained 

Since we have obtained the our final cost, constraints on which resources can and cant idle (ie which direction we need to move in workload space), we finally perform a minimization to find our optimum 'activities' over a fixed time horizon.

Once we have these optimum activities we then convert these activities into a probabilistic policy, and return decisions for the next t times steps.

# 0.A. Glossary
### Discrete Model

| Name | Desc |  Tex | Symbol | 
|:-----|:-----:| ---- | ------ |
|**Number of Buffers**| A scalar indicating the number of buffers in the system. | `\nbuff` | $$\nbuff \in \mathbb{Z_+}$$|
|**Number of Activities**| A scalar indicating the number of activities (or possible actions) in the system. | `\nact` | $$\nact \in \mathbb{Z_+}$$|
|**Number of Resources**| A scalar indicating the number of resources (or workers) in the system. | `\nres` | $$\nres \in \mathbb{Z_+}$$|
|**State**| A vector indicating the number of items in each buff in the system. | `\state` | $$\state \in \mathbb{Z}_+^{\nbuff}$$|
|**Arrival**| A vector indicating the arrivals arriving at each buffer at time step $t$ | `\arrival` | $$\arrival \in \mathbb{Z}_+^{\nbuff}$$|
|**Action**| A vector indicating all the actions taken by the whole system at time step $t$ | `\action` | $$\action \in \{0,1\}^{\nact}$$|
|**Constituency Matrix**| A binary matrix indicating the which actions each resource can perform simultaneously. <br><br> *eg: row $r$ indicates which actions resource $r$ can perform simultaneously*  | `\constituency` | $$\constituency \in \{0,1\}^{\nres \times \nact}$$|
|**Processing Matrix**| A matrix indicating the possible changes in buffer state for each action at time $t$. <br><br>  *eg: column $c$ indicates the changes in each buffer size at time $t$ due to action $c$*  | `\processing` | $$\processing \in \mathbb{Z}^{\nbuff \times \nact}$$|
|**Cost**| A vector indicating the cost associated with each buffer.| `\cost` | $$\cost : \mathbb{R}^{\nbuff}$$|
|**Cost Function**| A function mapping buffer state to a real value. This is usually linear | `\costfn` | $$\costfn : \mathbb{X}^{\nbuff} \to \mathbb{R} $$|

### Fluid Model
| Name | Desc |  Tex | Symbol | 
|:-----|:-----:| ---- | ------ |
|**Fluid State**| A continuous version of the state. | `\flstate` | $$\flstate \in \mathbb{R}_+^{\nbuff}$$|
|**Fluid Cumulative Activity**| A vector indicating the cumulative activity (time) each action has been worked on at time $t$. | `\flactivitycumul` | $$\flactivitycumul \in \mathbb{R}_+^{\nact}$$|
|**Fluid Activity Rate**| The instantaneous change in fluid cumulative activity with respect to time | `\flactivity` | $$\flactivity  := \dfrac{d\flactivitycumul}{dt}$$|

### Workload Space
| Name | Desc |  Tex | Symbol | 
|:-----|:-----:| ---- | ------ |
|**Workload Vector**| A vector corresponding to a resource, indicating how much it must work on each buffer to maximally drain the fluid | `\workload` | $$\workload \in \mathbb{R}^{\nbuff}$$|
|**Workload Matrix**| A matrix where each row corresponds to how much a resource must work on each buffer to maximally drain the fluid | `\workloadmat` | $$\workloadmat = [\workload_1^\T, ..., \workload_{\nres}^\T ], \workloadmat\in \mathbb{R}^{\nres \times \nbuff }$$|
|**Bottleneck**| An indicator vector signalling a resource, which corresponds to a particular workload vector| `\bottleneck` | $$\bottleneck \in \mathbb{R}_+^{\nres}$$|
|**Number of Workload Vectors**| A scalar indicating the number of workload vectors in our relaxation. | `\nwork` | $$\nwork < \nres, \nwork \in \mathbb{Z_+}$$|
|**Workload Matrix Approximator**|  A approximator matrix to $\workloadmat$,  keeping only the leading $\nwork$ workload $\workload$ vectors | `\workloadmathat` | $$\workloadmathat  = [\workload_1^\T, ..., \workload_{\nwork}^\T ], \workloadmathat \in \mathbb{R}^{\nwork \times \nbuff}$$|
|**Worktime Process**| A vector corresponding to time each buffer must be worked on (maximally) to drain the buffer to empty (assuming no arrivals)| `\worktime` | $$\worktime := \workloadmathat \state, \worktime \in \mathbb{R}_+^{\nwork}$$|
|**Load-Balance (Time) Vector**| A vector corresponding to time ('activity') each buffer must be worked on (maximally) to drain just arrivals| `\loadbalance` | $$\loadbalance := \workloadmathat \flarrival, \loadbalance \in \mathbb{R}_+^{\nwork}$$ |
|**Effective Cost**| A vector indicating the effective cost associated with the worktime process (of the bottlenecks).| `\effcost` | $$\effcost : \mathbb{R}^{\nbuff}$$|
|**Effective Cost Function**| A function mapping worktime process to a real value. This is piecewise-linear. | `\effcostfn` | $$\effcostfn : \mathbb{W}^{\nwork} \to \mathbb{R} $$|
|**Monotone Region**| The region of workspace where all components of effective cost level sets are non-negative. A step along any axis towards origin does not increase the effective cost. | `\monotone` | $$\monotone$$|
|**Idle Direction**| The unit direction pointing to the nearest point the monotone region that is reachable by idling | `\idledirection` | $$\idledirection$$|
|**Idling Set**| The set of indices referring to the bottlenecks that do not need to be constrained to non-idle/work flat out| `\idlingset` | $$\idlingset$$|
|**Non-Idling Set**| The set of indices referring to the bottlenecks that need to be constrained to non-idle/work flat out| `\nonidlingset` | $$\nonidlingset$$|


### Hedging Space
| Name | Desc |  Tex | Symbol | 
|:-----|:-----:| ---- | ------ |
|**Hedging Distance**| A scalar indicating the distance to nearest surface of the monotone region, which defines a hedging region | `\hedging` | $$\hedging \in \mathbb{R}$$|
|**Workload Matrix**| A matrix where each row corresponds to how much a resource must work on each buffer to maximally drain the fluid | `\workloadmat` | $$\workloadmat = [\workload_1^\T, ..., \workload_{\nres}^\T ], \workloadmat\in \mathbb{R}^{\nres \times \nbuff }$$|
|**Bottleneck**| An indicator vector signalling a resource, which corresponds to a particular workload vector| `\bottleneck` | $$\bottleneck \in \mathbb{R}_+^{\nres}$$|
|**Number of Workload Vectors**| A scalar indicating the number of workload vectors in our relaxation. | `\nwork` | $$\nwork \leq \nres, \nwork \in \mathbb{Z_+}$$|
|**Workload Matrix Approximator**|  A approximator matrix to $\workloadmat$,  keeping only the leading $\nwork$ workload $\workload$ vectors | `\workloadmathat` | $$\workloadmathat  = [\workload_1^\T, ..., \workload_{\nwork}^\T ], \workloadmathat \in \mathbb{R}^{\nwork \times \nbuff}$$|
|**Worktime Process**| A vector corresponding to time each buffer must be worked on (maximally) to drain the buffer to empty (assuming no arrivals)| `\worktime` | $$\worktime := \workloadmathat \state, \worktime \in \mathbb{R}_+^{\nwork}$$|
|**Load-Balance (Time) Vector**| A vector corresponding to time ('activity') each buffer must be worked on (maximally) to drain just arrivals| `\loadbalance` | $$\loadbalance := \workloadmathat \flarrival, \loadbalance \in \mathbb{R}_+^{\nwork}$$ |

# 1.A.1 Effective Cost Plotting Code


In [ ]:
### SURFACE PATH CODE
import numpy as np

def get_cbar(c_bars, w):
    f_cbar = np.array(c_bars)
    return c_bars[np.argmax(f_cbar @ w)]

def find_intersection(cbars, this_cbar, w, drift):
    f_cbar = np.array(cbars)
    diff = np.ma.array(f_cbar - this_cbar)
    lambdas = - diff @ w / (diff @ drift)
    valid = lambdas[np.where((lambdas > 0 ))[0]]
    
    return sorted(valid)[0] + 1e-10
        
def get_crossing_point(cbars, w, load):
    this_c_bar = get_cbar(cbars, w)
    is_monotone =  np.all(this_c_bar>=0)
    if is_monotone:
        drift = drift_vec(np.ones_like(w)*-1, load)
    else:
        drift = (this_c_bar < 0).astype(int) * load
    l = find_intersection(cbars, this_c_bar, w, drift)
    w_dot = w + l * drift 
    return w_dot, drift, is_monotone

def get_full_paths(w, cbars, load):
    s = w
    points = []
    is_monotone = np.all(get_cbar(cbars, w)>=0)
    while np.any(s > 0.0001) and not is_monotone:   
        e, d, is_monotone = get_crossing_point(cbars, s, load)
        points.append((s,e,d))
        s = e
    points.append((s,np.zeros(w.shape),d))
    return points

def drift_vec(v, load):
    direc = np.vstack([load, -(1-load)])
    return direc[(v < 0).astype(np.int64), np.arange(len(load))]

In [ ]:
## PLOTTING FUNCTIONS 

from mpl_toolkits.mplot3d import Axes3D

from matplotlib import cm
import matplotlib.pyplot as plt


DPI = 250
def plot_piecewise_workspace2D(cbars, points, dpi=250, contour_only=False, x_lim=None, y_lim=None):
    x_lim = [-2, 6] if x_lim is None else x_lim
    y_lim = [-2, 6] if y_lim is None else y_lim
    
    def f(x, y):
        f_cbar = np.array(cbars)
        v = np.array([x,y])
        return np.max(np.einsum( 'ij,jkl->ikl', f_cbar, v), axis = 0)

    fig = plt.figure(figsize=(10,5))

    ax = fig.add_subplot(1,2,1,projection='3d')

    xvalues = np.linspace(x_lim[0], x_lim[1], dpi)
    yvalues = np.linspace(y_lim[0], y_lim[1], dpi)

    xgrid, ygrid = np.meshgrid(xvalues, yvalues)
    zvalues = f(xgrid, ygrid)

    surf = ax.plot_surface(xgrid, ygrid, zvalues, rstride=5, cstride=5,linewidth=0, cmap=cm.plasma)
    
    if not contour_only:
        for x, y, style in points:
            plt.plot(x,y,f([x],[y])[0], style)
    ax.set_xlim(*x_lim)
    ax.set_ylim(*y_lim)
    ax.view_init(elev=30., azim=230)
       
    ax = fig.add_subplot(1,2,2)
    ax.set_xlim(*x_lim)
    ax.set_ylim(*y_lim)
    plt.contourf(xgrid, ygrid, zvalues, 30,cmap=cm.plasma)
    fig.colorbar(surf, aspect=18)
    plt.plot([0, 0], y_lim, '-.k', lw=0.3)
    plt.plot(x_lim, [0, 0], '-.k', lw=0.3)
    
    for x, y, style in points:
        plt.plot(x,y, style,lw=0.7)

    plt.tight_layout()
      
def get_w_star_annotations(w, w_star, colour='k'):
    w_plot = [[i] for i in w] + ['x'+colour]
    w_star_plot = [[i] for i in w_star] + ['*' + colour]
    return [w_plot, w_star_plot]    

def get_path_annotations(paths, colour='k'):
    annotations = []
    fmt = [':', '-.', '-', '--']
    for i, path in enumerate(paths):
        for s, e, _ in path:
            annotations.append([[si,ei] for si, ei in zip(s,e)] + [fmt[i] + colour])
    return annotations

def get_load_annotations(w, load):
    nonidling_direc = w - np.diag(1 - load)  
    idling_direc = w + np.diag(load) 
    annotations = []
    for ii, n in enumerate(nonidling_direc):
        arrow = [i for i in zip(n)] + ['+w']
        v = [[i,j] for i,j in zip(w, n)] + ['-w']
        annotations.extend([arrow, v])
    for jj, n in enumerate(idling_direc):
        arrow = [i for i in zip(n)] + ['+w']
        v = [[i,j] for i,j in zip(w, n)] + ['-w']
        annotations.extend([arrow, v])
    return annotations

def get_cost_for_line(cbars, start, end, drift):
    def f(x, y):
        f_cbar = np.array(cbars)
        v = np.array([x,y])
        return np.max(np.einsum( 'ij,jkl->ikl', f_cbar, v), axis = 0)
    dpi = int(np.abs(np.floor(100*(np.linalg.norm(end-start)/drift))))
    
    line = []
    for s, e in zip(start,end):
        line.append([np.linspace(s, e, dpi)])
    
    return f(*line)[0]

def line_drift_speed(start, end, load):
    return load @ (end-start)/np.linalg.norm(end-start)

def to_cumul(path, gamma=1):
    t1 = len(path)
    discount = [gamma**i for i in range(t1)]
    cumul = np.tril(np.ones((t1,t1)) * discount) @ path
    return cumul

def plot_paths(cbars, paths):
    fig = plt.figure(figsize=(10,5))
    ax1 = fig.add_subplot(1,2,1)
    ax2 = fig.add_subplot(1,2,2)
    gamma = 0.95
    
    x_max = 1000
    
    for lines in paths:
        path_points = []
        for start, end, load in lines:
            d = line_drift_speed(start, end, load)
            points = get_cost_for_line(cbars, start, end, d)
            
            path_points.append(points)
        path = np.hstack(path_points)
        ax1.plot(np.arange(len(path))/100, path)
        cumul = to_cumul(path, gamma)
        ax2.plot(np.arange(len(cumul[:x_max]))/100, cumul[:x_max],)
        


# 1.A.2 Hedging Plotting Code


In [ ]:
### HEDGING PATH CODE
import math 

def get_grads_of_intersection(cbars, monotone_point):
    monotone_cbar = get_cbar(cbars, monotone_point)
    neg = np.array([1,-1])
    diff =  (np.array(cbars) * neg) - (monotone_cbar * neg)
    diff = diff[~np.all(diff == 0, axis=1)]
    grads =  diff[:, 0 ] / diff[:, 1]
    return grads

def generate_noisy_changes(start, drift, grad, noise_scale, secs, dots_per_sec):
    dps = dots_per_sec
    points = secs * dps
    drift = drift / dps    
    
    noise = np.random.normal(scale=noise_scale, size=(2, points)) 
    policy = np.ones_like(noise) * drift[:, None] 
    coord_tot = np.array(start) 
    
    walk_change = (noise + policy)
    walk_change[:, 0] = np.array(start) 
    return walk_change

def reflect_noisy_changes(walk_change):
    ct = np.zeros(2)
    for i, c in enumerate(walk_change.T):
        ct += c
        if ct[1] - ct[0] * grad < 0:
            walk_change[1, i] += - (ct[1] - ct[0] * grad)
            ct[1] +=  -(ct[1] - ct[0] * grad)
    return walk_change

def generate_noisy_paths(start, drift, grad, noise_scale, secs, dots_per_sec):
    walk_change = generate_noisy_changes(start, drift, grad, noise_scale, secs, dots_per_sec)
    walk_change = reflect_noisy_changes(walk_change)
    return np.array([to_cumul(walk_change[0]), to_cumul(walk_change[1])])


def generate_illustrative_rw_paths(start, drift, grad, noise_scale, secs, dots_per_sec):
    noiseless_change = generate_noisy_changes(start, drift, grad, 1e-10, secs, dots_per_sec)
    noiseless =  np.array([to_cumul(noiseless_change[0]), to_cumul(noiseless_change[1])])
    walk_change = generate_noisy_changes(start, drift, grad, noise_scale, secs, dots_per_sec)
    unreflected =  np.array([to_cumul(walk_change[0]), to_cumul(walk_change[1])])
    walk_change2 = reflect_noisy_changes(walk_change)
    reflected =  np.array([to_cumul(walk_change2[0]), to_cumul(walk_change2[1])])
    return noiseless,  unreflected, reflected

def generate_many_illustrative_paths(samples, start, drift, grad, noise_scale, secs, dots_per_sec):
    rrw_p = []
    rw_p = []
    for i in range(samples):
        _, unref, ref = generate_illustrative_rw_paths(start, drift, grad, noise_scale, secs, dots_per_sec)
        rw_p.append(unref)
        rrw_p.append(ref)
    return np.array(rw_p), np.array(rrw_p)

def gradient_to_drift_scale(grad, drift):
    return - np.array([1,grad]) * np.linalg.norm(drift)/ np.linalg.norm([1, grad])


In [ ]:
### PLOTTING FUNCTIONS

def get_hedging_region_annotations(grads, beta_star, x_lim=None, y_lim=None):
    annotation = []
    x_lim = np.array([-2, 6]) if x_lim is None else np.array(x_lim)
    y_lim = np.array([-2, 6]) if y_lim is None else np.array(y_lim)
    for g, b in zip(grads, beta_star):
        
        cos = 1 / ((1 + g*g)**.5)
        sin = g / ((1 + g*g)**.5)
        
        annotation.append([x_lim, g*x_lim + b/cos, '--w'])
        annotation.append([x_lim, g*(x_lim), ':w'])
    return annotation


def plot_cumul_cost(f_cbar, paths, gamma):
    fig = plt.figure(figsize=(10,5))
    ax1 = fig.add_subplot(1,2,1)
    ax2 = fig.add_subplot(1,2,2)
    x_max = 1000
    
    for lines in paths:
        path = f_cbar(np.vstack(lines))
        ax1.plot(np.arange(len(path))/100, path)
        cumul = to_cumul(path, gamma)
        ax2.plot(np.arange(len(cumul[:x_max]))/100, cumul[:x_max],)
    ax1.legend(-0.1 * np.arange(len(paths)))
    ax2.legend(-0.1 * np.arange(len(paths)))

def get_noisy_path_annotations(grad, drift, beta_stars, noisy_path, fmts=['-', '--', '-.']):
    annotation = []

    cos = 1/ ((1 + grad*grad)**.5)  
    
    for fmt, beta_star in zip(fmts,beta_stars):
        x_path_hedge = noisy_path[0]
        y_path_hedge = noisy_path[1] + beta_star/cos
        annotation.append([x_path_hedge, y_path_hedge, fmt + 'k'])
    return annotation

In [ ]:
def plot_3_example_walks(cbars, load):
    
    x_lim = [0, 10]
    y_lim = [0, 10]

    # find gradients of intersection w monotone region -> define affine hedging regions
    monotone_point = [4,4]
    grads = get_grads_of_intersection(cbars, monotone_point) 
    annotations = [get_hedging_region_annotations(grads, [0,0], x_lim, y_lim)  for _ in range(3)] 

    noise = 0.25
    drift = - (1 - load)
    grad = grads[-1] # lower surface gradient and beta star
    drift_g = gradient_to_drift_scale(grad, drift) # make a drift vector out of gradient 

    start = [8,  8 * grad] # start on intersection
    seconds = 20
    noise_per_second = 3
    for a in annotations:
        illustrative_paths = generate_illustrative_rw_paths(start, drift_g, grad, noise, seconds, noise_per_second)
        path_annotations = []
        for ip, fmt in zip(illustrative_paths, ['-', '--', '-']):
            path_annotations += get_noisy_path_annotations(grad, drift_g, [0], ip, [fmt])
        a += path_annotations
        a += get_load_annotations([0,0], load)

    gamma = 0.95

    plot_piecewise_contours(cbars, annotations, gamma, x_lim=x_lim, y_lim=y_lim)
    
def plot_piecewise_contours(cbars, paths, gamma, dpi=250, x_lim=None, y_lim=None):
    x_lim = [-2, 6] if x_lim is None else x_lim
    y_lim = [-2, 6] if y_lim is None else y_lim
    
    def f(x, y):
        f_cbar = np.array(cbars)
        v = np.array([x,y])
        return np.max(np.einsum( 'ij,jkl->ikl', f_cbar, v), axis = 0)

    cols = len(paths)
    rows = 2
    fig = plt.figure(figsize=( 9.5, rows * 9.5/cols))


    xvalues = np.linspace(x_lim[0], x_lim[1], dpi)
    yvalues = np.linspace(y_lim[0], y_lim[1], dpi)

    xgrid, ygrid = np.meshgrid(xvalues, yvalues)
    zvalues = f(xgrid, ygrid)
    
    plots = len(paths)
    for i, p in enumerate(paths):
        ax1 = fig.add_subplot(rows,cols,i+1)

        surf = plt.contourf(xgrid, ygrid, zvalues, 30, cmap=cm.plasma)
        plt.plot([0, 0], y_lim, '-.k', lw=0.3)
        plt.plot(x_lim, [0, 0], '-.k', lw=0.3)
        for x, y, style in p:
            plt.plot(x,y, style,lw=0.7)

        ax1.set_xlim(*x_lim)
        ax1.set_ylim(*y_lim)
        fig.colorbar(surf, aspect=18, orientation='horizontal')

    for i, p in enumerate(paths):
        ax2 = fig.add_subplot(rows,cols,i+1 + cols)
        for x, y, style in p[4:7]:
            path = f_cbar(np.vstack([x,y]))
            cumul = to_cumul(path, gamma=gamma)
            ax2.plot(np.arange(len(cumul)), cumul,)
    
    plt.tight_layout()

In [ ]:
def plot_distribs(cbars, dist, annotations, all_paths, gamma, dpi=250, x_lim=None, y_lim=None):
    x_lim = [-2, 6] if x_lim is None else x_lim
    y_lim = [-2, 6] if y_lim is None else y_lim
    
    def f(x, y):
        f_cbar = np.array(cbars)
        v = np.array([x,y])
        return np.max(np.einsum( 'ij,jkl->ikl', f_cbar, v), axis = 0)

    cols = 3
    rows = 1
    fig = plt.figure(figsize=( 9.5, rows * 9.5/cols))

    xvalues = np.linspace(x_lim[0], x_lim[1], dpi)
    yvalues = np.linspace(y_lim[0], y_lim[1], dpi)

    xgrid, ygrid = np.meshgrid(xvalues, yvalues)
    zvalues = f(xgrid, ygrid)
    
    plots = len(paths)
    for i in range(2):
        ax1 = fig.add_subplot(rows,cols,i+1)
        surf = plt.contourf(xgrid, ygrid, zvalues, 30, cmap=cm.plasma)
        
        plt.plot([0, 0], y_lim, '-.k', lw=0.3)
        plt.plot(x_lim, [0, 0], '-.k', lw=0.3)
        for x, y, style in annotations:
            plt.plot(x,y, style,lw=0.7)
            
        plt.plot(*dist[i][0],  '-k', lw=0.7)
        plt.plot(*dist[i][1],  '--k', lw=0.7)
        plt.plot(*dist[i][2],  '--k', lw=0.7)
        ax1.set_xlim(*x_lim)
        ax1.set_ylim(*y_lim)
        fig.colorbar(surf, aspect=18, orientation='horizontal')
        
    ax3 = fig.add_subplot(rows,cols,3)
    bins = 50
    Z = np.zeros((bins+1,bins+1))
    x1 = np.arange(bins + 2) * (x_lim[1] - x_lim[0])/bins
    y1 = np.arange(bins + 2) * (y_lim[1] - y_lim[0])/bins
    plt.plot([0, 0], y_lim, '-.k', lw=0.3)
    plt.plot(x_lim, [0, 0], '-.k', lw=0.3)
    for x, y, style in annotations:
        plt.plot(x,y, style,lw=0.7)
    
    for i in range(all_paths.shape[2]):
        H, xb, yb = np.histogram2d(all_paths[:, 0, i], all_paths[:, 1, i], bins=np.array([x1, y1]))
        Z += H * (gamma ** i)
    Z /= (all_paths.shape[0] * all_paths.shape[2])

    xm, ym = np.meshgrid(xb[:-1], yb[:-1])
    surf = plt.contourf(xgrid, ygrid, zvalues, 30, alpha=0.5, cmap=cm.Greys)

    y1 = np.linspace(y_lim[0], y_lim[1], 100)
    surf = plt.contourf(xm, ym, Z.T, 30, alpha=.8,  cmap=cm.plasma)
    
    ax3.set_xlim(*x_lim)
    ax3.set_ylim(*y_lim)
    fig.colorbar(surf, aspect=18, orientation='horizontal')

    plt.tight_layout()

In [ ]:
     
def plot_distributions_walks(cbars, drift, noise, gamma):
    samples = 1000
        
    x_lim = [0, 10]
    y_lim = [0, 10]

    # find gradients of intersection w monotone region -> define affine hedging regions
    monotone_point = [4,4]
    grads = get_grads_of_intersection(cbars, monotone_point) 
    annotations = get_hedging_region_annotations(grads, [0,0], x_lim, y_lim)
    annotations += get_load_annotations([0,0], load)

    start = [8,  8 * grad] # start on intersection
    seconds = 30
    noise_per_second = 5
    
    rw, rrw = generate_many_illustrative_paths(samples, start, drift, grad, noise, seconds, noise_per_second)
    
    mean_rw = np.mean(rw, axis=0)
    q1_rw =np.quantile(rw, 0.25, axis=0)
    q3_rw = np.quantile(rw, 0.75, axis=0)
    
    mean_rrw = np.mean(rrw, axis=0)
    q1_rrw =np.quantile(rrw, 0.25, axis=0)
    q3_rrw = np.quantile(rrw, 0.75, axis=0)

    rw_dist = [mean_rw, q1_rw, q3_rw]
    rrw_dist = [mean_rrw, q1_rrw, q3_rrw]

    dist = [rw_dist, rrw_dist]
    plot_distribs(cbars, dist, annotations, rrw, gamma, x_lim=x_lim, y_lim=y_lim)
    
    


### 3 Hedging 

Now that we have the transformation matrix $\workloadmathat$, our problem optimisation can be expressed in workload space:

$$\bar{c}(\worktime) = \min_{x}  \cost^\T \state \text{ s.t. } \workloadmathat \state = \worktime $$

where the minimisation over $\state$ for a fixed $\worktime$ arises due to the multiple possible solutions in to $ \workloadmathat \state = \worktime$.

To solutions to the this equation are piecewise linear, as can be seen by rewriting the equation  


The cost is piecewise linear and convex. 

1. Find $w = \hat{\Xi}x$
2. Find the nearest face of the closest face of monotone regions
    1. This looks line a sort of line search method. 
        * Why two vectors?
        * Do we have to try all faces?
    2. Find the coordinate direction with the shortest distance to this plane ("projection")
3. Compute asymptotic covariance of the workload-space random walk
4. 

1. Find $w = \hat{\Xi}x$
2. Find the nearest face of the closest face of monotone regions
    1. This looks line a sort of line search method. 
        * Why two vectors?
        * Do we have to try all faces?
    2. Find the coordinate direction with the shortest distance to this plane ("projection")
3. Compute asymptotic covariance of the workload-space random walk
4. 

# B. Duality and Optimisation



-----
*Aside*

Although not a formal proof, this duality can be seen from the Lagrangian formulation of the former:

$$T* = \max_{\workload, \bottleneck \geq \mathbf{0}} \min T + \workload^\T(   \flprocessing \flactivitycumulfree +\mathbf{q}_1 -\mathbf{q}_2   ) + \bottleneck^\T(  \constituency \flactivitycumulfree - \mathbf{T}) + [\epsilon_1^\T(-\flactivitycumulfree) + \epsilon_2 (-T)]$$ 

where $\workload, \bottleneck$ are Lagrange multipliers or KKT variables. Note that due to the inequality conditions, $\bottleneck$ is non-negative. This is very similar to the latter's Lagrangian formulation, with  $\flactivitycumulfree$ and $T$ as KKT variables:

$$ \min_{\flactivitycumulfree \geq \mathbf{0}, T\geq 0} \max\workload^\T( \mathbf{q}_1 -  \mathbf{q}_2) + \flactivitycumulfree^\T(\flprocessing^\T \workload + \constituency^\T\bottleneck) + T(-\mathbf{1}^\T\bottleneck + 1 ) + [\epsilon_1^\T \bottleneck]$$

## Defining Your Network

$\constituency $



### 3 Hedging 

Now that we have the transformation matrix $\workloadmathat$, our problem optimisation can be expressed in workload space:

$$\bar{c}(\worktime) = \min_{x}  \cost^\T \state \text{ s.t. } \workloadmathat \state = \worktime $$

where the minimisation over $\state$ for a fixed $\worktime$ arises due to the multiple possible solutions in to $ \workloadmathat \state = \worktime$.

To solutions to the this equation are piecewise linear, as can be seen by rewriting the equation  


The cost is piecewise linear and convex. 

1. Find $w = \hat{\Xi}x$
2. Find the nearest face of the closest face of monotone regions
    1. This looks line a sort of line search method. 
        * Why two vectors?
        * Do we have to try all faces?
    2. Find the coordinate direction with the shortest distance to this plane ("projection")
3. Compute asymptotic covariance of the workload-space random walk
4. 

$$\min_{\mathbf{z}, T} T  \\
\\\text{ s.t.} 
\\ \mathbf{q}_2 = \mathbf{q}_1 + \flprocessing \flactivitycumulfree  \\
\constituency \flactivitycumulfree \leq \mathbf{T} \\
\flactivitycumulfree \geq \mathbf{0} \\
T \geq 0 
$$

to which the equivalent dual is: 

$$\max_{\workload, \bottleneck} \workload^\T( \mathbf{q}_1 -  \mathbf{q}_2)
\\\text{ s.t.} 
\\   \flprocessing^\T \workload + \constituency^\T\bottleneck \geq \mathbf{0}  
\\- \mathbf{1}^\T\bottleneck + 1 \geq 0 
\\ \bottleneck \geq \mathbf{0}
$$


If we let  $\mathbf{q}_1 = \flarrival$, $\mathbf{q}_2 = 0$ - i.e we drain to empty from an initial state of $\flarrival$ - we get the succinct linear optimisation to find our $\workload$.

$$\min_{\workload, \bottleneck} \workload^T\flarrival\\
\text{ s.t.} \\  - \flprocessing^T \workload - \constituency^T\bottleneck \leq \mathbf{0}  \\
\mathbf{1}^T\bottleneck \leq 1 \\
 \bottleneck \geq \mathbf{0}
$$

